# Questions to ask before you start:

1. Do you have a file of the input poses ready? Does it contain the table of encoder info on the bottom right, and groundskeeping data in the top left?

## Did the script fail or give an error?
#### The most common cause of errors is if one of the input files is not formatted properly. 
1. The first thing I would check is the input pose spreadsheet. 
 * If that spreadsheet was created by a human and not this program, there's a chance it was formatted incorrectly and you'll have to do some copy/pasting into the "Poses - template.xlsx" file.
2. The second thing I would check is the "FDPR Shifts" file
 * I find the easiest way to proceed is to copy/paste the relevant lines from the spreadsheet that the FDPR guys provide into the template file. Remember that only the columns in yellow are the mandatory ones.

## 0 Parameter definitions

### 0.0.1 Parameters you'll need to tweak

In [1]:
baseline_fname = 'files/Poses - Practice Input.xlsx' #What is the name of the input file? Don't forget to add "files/" if it's located in that subdirectory
#Name of the tab in the excel file that contains the data for your baseline poses. 
#If you're using a spreadsheet that was generated using this code, you'll want to set sheetname='update'. 
#If you're using a spreadsheet that came from Manal, chances are the tab is named 'input for Tyler analysis' or something
#sheetname = 'update'
sheetname = 'Input for Tyler analysis'

#What do you want to name the output file?
#NOTE, the output spreadsheet from this code should not be used as an input for 'Pose Updater - FDPR.ipynb'.
output_filename = 'Poses - FDPR Practice Output.xlsx' #What do you want to name the output file? 
#Remember the desired format is "Poses - [name of new circuit, i.e. BA2C3] FDPR update from [name of baseline circuit, i.e. BA2C1] - [Your name].xlsx"

psf_shift_filename = 'files/FDPR shifts - template.xlsx' #File that contains the PSF shift data
psf_shift_sheetname= 'PSF Shifts' #Name of the sheet within the file that cotains the sheetname

---
# Below here be math
---
### 0.1 Ingesting values from the template

In [2]:
%matplotlib notebook
import os
from adm_library import *

In [3]:
spreadsheet = pd.read_excel(baseline_fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
sMPA_angle_to_WCS_deg.name = 'sMPA XYZ Euler angle to WCS'
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(baseline_fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:S')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]

#Depending on the origin of the input spreadsheet, the desired encoder values are in a different place.
if sheetname == 'Input for Tyler analysis':
    pose_encoders = spreadsheet.iloc[0:5,7:13]
elif sheetname == 'update':
    pose_encoders = spreadsheet.iloc[0:5,12:18]
    
#Rename the columns because by default, each column header got a ".1" added to the end
new_columns=[]
for column in pose_encoders.columns:
    new_columns.append(column[:-2])
pose_encoders.columns = new_columns
pose_encoders = pose_encoders.set_index('Name').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     0.943594   1.280695  -0.338386  0.031453   0.035137
PR2  -109.505749 -45.867074  22.370475 -4.911732  10.680266
PR3   111.309055 -45.373337  22.058458 -4.857077 -10.600321
PR4   103.076801  59.765562  30.696044  5.307985 -10.311998
PR5   -101.43058   59.40225  30.809096  5.269783  10.410043 

Provided 5DOF encoder values: 
                                                            
Name                                                       
PR1     0.915828   1.315707  -0.285326  0.033599   0.035898
PR2  -109.475990 -45.798775  22.414907 -4.904663  10.683385
PR3   111.188948 -45.262171  22.183694 -4.862801 -10.596318
PR4   102.952223  59.823353  30.901985  5.303956 -10.315476
PR5  -101.416216  59.420073  30.939794  5.278220  10.405904


C:\Users\ebray\AppData\Local\Temp\ipykernel_2680\93341823.py:11: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  spreadsheet = pd.read_excel(baseline_fname,


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [4]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [5]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

### 0.3 Form a dataframe with the values supplied so far

In [6]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA',['Rx','Ry','Rz']] = sMPA_angle_to_WCS_deg.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,Rz,uvec_X,uvec_Y,uvec_Z
Name,,,,,,,,,,
PR1,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,NaN,0.792405,248.143393,1268.071794
PR2,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,NaN,239.466788,350.312004,1220.458522
PR3,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,NaN,-237.694885,349.239187,1221.112189
PR4,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,NaN,-231.300660,128.369032,1264.754223
PR5,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,NaN,233.475701,129.171851,1264.272746
sMask,1.3107,116.244531,587.154,NaN,NaN,g,NaN,NaN,NaN,NaN
sMPA,1.736,249.4149,1267.7131,-24.285640,0.023570,purple,0.13521,0.000411,0.411286,0.911506


# 1 Updating poses based on results from FDPR

### 1.1 Reading in offset data from the FDPR team's spreadsheet.

In [7]:
#These data will be used to update the position of the calculated endpoints in the sMPA frame
shift_data = pd.read_excel(psf_shift_filename,sheet_name=psf_shift_sheetname,skiprows=0,usecols='C:K')
shift_data = shift_data.iloc[0:5]
shift_data.set_index('camera',inplace=True)
print(shift_data)

       det normal dz (um)  chief ray dz (um)    row  ...  dy (px)  dx (um)  dy (um)
camera                                               ...                           
PR1                   -87              -89.0  261.2  ...     -5.2    -51.0    -52.0
PR3                  -250             -258.0  260.0  ...     -4.0   -149.0    -40.0
PR4                   -13              -14.0  264.9  ...     -8.9    -71.0    -89.0
PR5                   -19              -21.0  268.3  ...    -12.3    -81.0   -123.0
PR2                  -137             -141.0  252.1  ...      3.9   -122.0     39.0

[5 rows x 8 columns]


#### 1.1.1 Calculate some transformation variables necessary for converting between the 5DOF and sMPA frame

In [8]:
translation_to_sMPA = df.loc['sMPA',['X','Y','Z']]
rotation_from_sMPA_to_5DOF = R.from_matrix(rotmat_from_2vec(np.array([0,0,1]),df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']].values.astype(float)))

### 1.2 Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 6DOF optimization)

In [10]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results_v2(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1  0.050932  0.064591  0.065888
PR2  0.148223  0.002920  0.149056
PR3  0.101444  0.106489  0.227169
PR4  0.068284  0.082655 -0.022996
PR5  0.084504  0.114375 -0.030150 

Best-fit 6DoF deltas to apply to real-space poses: 
 [-0.0535  1.1847 -0.1361  0.051   0.0055 -0.0051] 

Residuals after incorporating the best-fit 6DoF shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.042    0.008   -0.018
PR2    0.048   -0.068   -0.004
PR3    0.002   -0.019    0.031
PR4   -0.012    0.001   -0.023
PR5    0.004    0.078    0.014 

endpt_X    0.029241
endpt_Y    0.047092
endpt_Z    0.020096
dtype: float64


### 1.3 Convert new best-fit poses to encoder space

In [11]:
df_after_fitting_FDPR_shifts_in_encoder_space = pd.DataFrame(columns=['X','Y','Z','Rx','Ry'],index=df_after_fitting_FDPR_shifts.index)
for pose in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
    df_after_fitting_FDPR_shifts_in_encoder_space.loc[pose] = calculate_encoders_from_5DOF(pd.DataFrame(df_after_fitting_FDPR_shifts.loc[pose]).T).squeeze()

print('New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: \n',df_after_fitting_FDPR_shifts_in_encoder_space,'\n')
print('Baseline pose encoders (displayed again here just for reference): \n', pose_encoders)

New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
               X          Y          Z        Rx         Ry
PR1    0.863147   2.475074  -0.467852  0.083156   0.043714
PR2 -109.459218 -44.795394  22.159519 -4.859743   10.68621
PR3  111.203334 -43.901999  22.032123  -4.80542 -10.593124
PR4  102.826285  61.150819  30.775897  5.362975 -10.303491
PR5 -101.537535  60.392291  30.713764  5.321132  10.416939 

Baseline pose encoders (displayed again here just for reference): 
                                                            
Name                                                       
PR1     0.915828   1.315707  -0.285326  0.033599   0.035898
PR2  -109.475990 -45.798775  22.414907 -4.904663  10.683385
PR3   111.188948 -45.262171  22.183694 -4.862801 -10.596318
PR4   102.952223  59.823353  30.901985  5.303956 -10.315476
PR5  -101.416216  59.420073  30.939794  5.278220  10.405904


# 2 Write the new poses to an Excel file in the current directory

In [12]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel(output_filename,'',update_type='FDPR',baseline_filepath=baseline_fname,
                             columns=columns,sMPA_angle_to_WCS_deg=sMPA_angle_to_WCS_deg,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             rigid_body_correction=best_fit_deltas,
                             df=df,df_encoders=pose_encoders,df_update=df_after_fitting_FDPR_shifts,df_update_encoders=df_after_fitting_FDPR_shifts_in_encoder_space,
                            focal_length=focal_length)

**Writing to Excel complete.**
**Filename:  Poses - FDPR Practice Output.xlsx
